In [1]:
import numpy as np
import fiona
import pprint
from madmex.models import PredictObject
from madmex.models import PredictClassification, Region, ValidClassification
from madmex.util import chunk
import json
from shapely.geometry import mapping
from madmex.util.spatial import geometry_transform
from shapely.geometry import shape
from dask.distributed import Client
import os
from operator import itemgetter
from django.contrib.gis.geos.geometry import GEOSGeometry
from fiona.crs import to_string

In [30]:
predict_name = 'lc_rf_s1_2_20m_resampled_10m_Jalisco_recipe_2017'

In [3]:
qs_ids = PredictClassification.objects.filter(name=name_predict).distinct('predict_object_id')

In [4]:
len(qs_ids)

56

In [5]:
list_ids = [x.predict_object_id for x in qs_ids]

In [6]:
region = 'Jalisco'

In [7]:
region_geom = Region.objects.get(name=region).the_geom
region_geojson = region_geom.geojson
geometry = json.loads(region_geojson)
proj4='+proj=lcc +lat_1=17.5 +lat_2=29.5 +lat_0=12 +lon_0=-102 +x_0=2500000 +y_0=0 +a=6378137 +b=6378136.027241431 +units=m +no_defs'
geometry_region_proj = geometry_transform(geometry,proj4)

In [70]:
#region_geojson

In [61]:
geometry

{'type': 'Polygon',
 'coordinates': [[[2227660.0, 1026160.0],
   [2227660.0, 976140.0],
   [2277680.0, 976140.0],
   [2277680.0, 1026160.0],
   [2227660.0, 1026160.0]]]}

In [8]:
list_ids[0]

169

In [9]:
len(list_ids)

56

In [10]:
validation_set='validation_Jalisco'

In [11]:
list_qs = []
list_seg_paths = []
proj4_out = '+proj=longlat'
proj4_in = '+proj=lcc +lat_1=17.5 +lat_2=29.5 +lat_0=12 +lon_0=-102 +x_0=2500000 +y_0=0 +a=6378137 +b=6378136.027241431 +units=m +no_defs'

In [12]:
for id in list_ids:
    seg = PredictObject.objects.filter(id=id)
    path = seg[0].path
    poly = seg[0].the_geom
    poly_geojson = poly.geojson
    geometry = json.loads(poly_geojson)
    geometry_proj = geometry_transform(geometry,proj4_out,crs_in=proj4_in)
    poly_proj = GEOSGeometry(json.dumps(geometry_proj))
    query_set = ValidClassification.objects.filter(valid_object__the_geom__contained=poly_proj,
                                               valid_set=validation_set).prefetch_related('valid_object', 'valid_tag')
    list_qs.append(query_set)
    list_seg_paths.append(path)

In [13]:
list_qs

[<QuerySet [<ValidClassification: ValidClassification object (268)>, <ValidClassification: ValidClassification object (367)>, <ValidClassification: ValidClassification object (385)>]>,
 <QuerySet []>,
 <QuerySet []>,
 <QuerySet [<ValidClassification: ValidClassification object (301)>, <ValidClassification: ValidClassification object (302)>, <ValidClassification: ValidClassification object (460)>, <ValidClassification: ValidClassification object (461)>, <ValidClassification: ValidClassification object (473)>]>,
 <QuerySet [<ValidClassification: ValidClassification object (468)>]>,
 <QuerySet [<ValidClassification: ValidClassification object (519)>, <ValidClassification: ValidClassification object (520)>]>,
 <QuerySet [<ValidClassification: ValidClassification object (324)>, <ValidClassification: ValidClassification object (327)>, <ValidClassification: ValidClassification object (328)>, <ValidClassification: ValidClassification object (337)>, <ValidClassification: ValidClassification obj

In [14]:
qs_dc_tile = list_qs[0]

In [15]:
qs_dc_tile

<QuerySet [<ValidClassification: ValidClassification object (268)>, <ValidClassification: ValidClassification object (367)>, <ValidClassification: ValidClassification object (385)>]>

In [16]:
seg_path_s3 = list_seg_paths[0]

In [17]:
qs_element_dc_tile = qs_dc_tile[0]

In [18]:
qs_element_dc_tile.valid_tag.id

8

In [31]:
id_dc_tile = list_ids[0]

In [19]:
def valid_object_to_feature(x, crs):
    attr = {'class': x.valid_tag.numeric_code}
    if crs is None:
        geometry = json.loads(x.valid_object.the_geom.geojson)
    else:
        geometry = json.loads(x.valid_object.the_geom.transform(crs, clone=True).geojson)
    feature = {
        "type": "Feature",
        "geometry": geometry,
        "properties": attr
    }
    return feature

In [20]:
shape_region=shape(geometry_region_proj)

In [92]:
with fiona.open(seg_path_s3) as src:
    crs = to_string(src.crs)
    fc_qs = [valid_object_to_feature(x, crs) for x in qs_dc_tile]
    fc_qs_in_region = [(mapping(shape_region.intersection(shape(x['geometry']))),
                    x['properties']['class']) for x in fc_qs if shape_region.intersects(shape(x['geometry']))]
    fc_qs = None
    pred_objects_sorted = PredictClassification.objects.filter(name=predict_name,
                                                               predict_object_id=id_dc_tile).prefetch_related('tag').order_by('features_id')
    fc_pred=[(x['properties']['id'], x['geometry']) for x in src]
    fc = None
    fc_pred_sorted = sorted(fc_pred, key=itemgetter(0))
    fc_pred = [(x[0][1], x[1].tag.numeric_code) for x in zip(fc_pred_sorted, pred_objects_sorted)]
    fc_pred_sorted = None
    pred_objects_sorted = None
    fc_pred_intersect_validset = [(x[0],x[1]) for x in fc_pred for y in fc_qs_in_region if shape(x[0]).intersects(shape(y[0]))]
    fc_pred = None

In [101]:
from madmex.validation import prepare_validation

In [ ]:
y_true, y_pred, sample_weight = prepare_validation(fc_qs_in_region, fc_pred_intersect_validset)